In [2]:
!pip install -q -q -q geopandas
!pip install -q -q -q rasterio
!pip install -q -q -q py7zr

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import rasterio
from rasterio import features

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KDTree
from sklearn.neighbors import RadiusNeighborsRegressor

Chargement des fonctions et des données

In [2]:
def rasterizeBlackWhite(sdf, size, points=True):
    """sdf : a geopandas DataFrame
       size : resolution
       points : if points is true return a pandas DataFrame, otherwise a numpy array"""
    bounds = sdf.total_bounds
    geometry = sdf.geometry
    xmin = np.floor(bounds[0] / size) * size
    ymin = np.floor(bounds[1] / size) * size
    xmax = np.ceil(bounds[2] / size) * size
    ymax = np.ceil(bounds[3] / size) * size
    geometry = geometry.translate(xoff=-xmin, yoff=-ymin, zoff=0.0)
    geometry = geometry.scale(xfact=1/size,yfact=1/size,origin=(0,0))
    width = int((xmax - xmin)/size)
    height = int((ymax - ymin)/size)
    raster = features.rasterize(geometry, (height,width),fill=0,all_touched=True)
    if points==True:
        y, x = np.where(raster) 
        return(pd.DataFrame({'x' : x * size + xmin , 'y' : y * size + ymin}))
    else:
        return(np.flip(raster.reshape(raster.shape[0], raster.shape[1]),0))

In [18]:
def neighbors_near_point(df, list_of_radius):
    neighbors = np.zeros([df.shape[0],list_of_radius.size],dtype=np.float)
    tree = KDTree(df[['x', 'y']]) 
    dist, ind = tree.query(df[['x', 'y']], np.max(list_of_radius))   
    for i, r in enumerate(list_of_radius):
        neighbors[:,i] =list(map(lambda x : np.sum(x <= r), dist)) 
    result = pd.DataFrame(neighbors)
    result.columns = [str(t) for t in list_of_radius]
    return(pd.concat([df[['x', 'y']].reset_index(), result], axis=1))

In [4]:
bocage = gpd.read_file('building_bocage.gpkg')
openfield = gpd.read_file('building_openfield.gpkg')

### Rasterization des fonds à 4 m

In [19]:
r_openfield = rasterizeBlackWhite(openfield, 4)

In [6]:
r_bocage = rasterizeBlackWhite(bocage, 4)

### Calcul des voisinages

In [ ]:
neigh_openfield = neighbors_near_point(r_openfield, np.array([25,50,100,200,400,800,1600])) 

In [ ]:
neigh_bocage = neighbors_near_point(r_bocage, np.array([25,50,100,200,400,800,1600])) 

### Calcul de la GWFA en un point

In [ ]:
def Gwfa_en_un_point(df, x_centre, y_centre, q):
    distance = (df.x - x_centre)** 2 + (df.y - y_centre)**2
    weight = (1 - (distance / 16000**2))**2
    


In [8]:
df = r_openfield.copy()
list_of_radius = np.array([25,50,100,200,400,800,1600])

In [9]:
neighbors = np.zeros([df.shape[0],list_of_radius.size],dtype=np.float)

In [10]:
tree = KDTree(df[['x', 'y']]) 

In [12]:
dist, ind = tree.query(df[['x', 'y']], np.max(list_of_radius))   

In [17]:
for i, r in enumerate(list_of_radius):
    neighbors[:,i] =list(map(lambda x : np.sum( x <= r), dist)) 